# Exercises for Chapter 4

In [11]:
import numpy as np
import matplotlib.pyplot as plt

import navigation_utilities as navutil

## Chapter 4, Exercise 1

### Effect of 1° error on position estimation

### A. Spherical Earth model

In [12]:
# Given quantities
ANGLE_ERROR_DEG = 1.
SPH_EARTH_RADIUS_KM = 6371.

# Specify the observer latitude
OBS_LATITUDE_DEG = 36.1627

# Convert angles to radians
angle_error_rad = ANGLE_ERROR_DEG * np.pi / 180.0
obs_latitude_rad = OBS_LATITUDE_DEG * np.pi / 180.0

# Calculate the arc length of the error
distance_error_km = SPH_EARTH_RADIUS_KM * angle_error_rad * np.cos(obs_latitude_rad)

print(f'An error of 1° longitude at latitude {OBS_LATITUDE_DEG}° on a '
      f'spherical Earth results in a position error of '
      f'{distance_error_km * 1000} m')

An error of 1° longitude at latitude 36.1627° on a spherical Earth results in a position error of 89772.62691196235 m


### B. WGS 84 Ellipsoid model

In [13]:
# WGS 84 equatorial radius of Earth
WGS84_EQ_EARTH_RADIUS_M = 6378137.0

# Obtain the eccentricity from the reciprocal flattening
REC_FLATTENING = 298.257223563
flattening = 1.0/REC_FLATTENING
eccentricity = np.sqrt(2*flattening - flattening**2)

# Obtain the geocentric latitude from the observer geodetic latitude
obs_lat_geoc_rad = np.arctan((1 + eccentricity**2) * np.tan(obs_latitude_rad))

# Calculate the arc length of the error
distance_error_m = WGS84_EQ_EARTH_RADIUS_M * angle_error_rad * np.cos(obs_lat_geoc_rad)

print(f'An error of 1° longitude at latitude {OBS_LATITUDE_DEG}° on a '
      f'WGS 84 ellipsoidal Earth results in a position error of '
      f'{distance_error_m} m')

An error of 1° longitude at latitude 36.1627° on a WGS 84 ellipsoidal Earth results in a position error of 89663.73674431579 m


## Chapter 4, Exercise 2

### Palo Alto Runway

In [14]:
# Given coordinates
RWY_30_START_M = np.array([-2694685.473, -4293642.366, 3857878.924])
RWY_30_END_M = np.array([-2694892.460, -4293083.225, 3858353.437])

### A. WGS 84 geodetic LLA of Runway 30 start location

In [15]:
rwy_30_start_lla = navutil.wgs_lla_from_xyz(RWY_30_START_M)

print(f'Runway 30 starts at {rwy_30_start_lla[0]}° longitude, '
      f'{rwy_30_start_lla[1]}° latitude, and '
      f'{rwy_30_start_lla[2]} m altitude according to the WGS 84 ellipsoid.')

Runway 30 starts at -122.11233899586935° longitude, 37.45837643292903° latitude, and -31.455705165348213 m altitude according to the WGS 84 ellipsoid.


### B. Latitude and Longitude in Degrees, Arc-Minutes, and Arc-Seconds

In [16]:
lon_dms = navutil.degree_min_sec_from_decimal(rwy_30_start_lla[0])
lat_dms = navutil.degree_min_sec_from_decimal(rwy_30_start_lla[1])

print(f'Runway 30 starts at longitude {lon_dms[0]}°, {lon_dms[1]} minutes, '
      f'{lon_dms[2]} seconds and latitude {lat_dms[0]}°, {lat_dms[1]} '
      f'minutes, {lat_dms[2]} seconds.')

Runway 30 starts at longitude -122.0°, 6.0 minutes, 44.0 seconds and latitude 37.0°, 27.0 minutes, 30.0 seconds.


### C. Height of the start point above the local Ellipsoid

In [17]:
PALO_ALTO_GEOID_HEIGHT_M = -33.0
rwy30_start_alt_pa_geoid = rwy_30_start_lla[2] - PALO_ALTO_GEOID_HEIGHT_M

print(f'Runway 30 starts {rwy30_start_alt_pa_geoid} m above the local '
      f'Palo Alto geoid.')

FAA_PUB_RWY30S_HEIGHT_MSL = 3.0
print(f'This is {np.abs(rwy30_start_alt_pa_geoid-FAA_PUB_RWY30S_HEIGHT_MSL)} '
      f'm off from the published FAA altitude relative to Mean Sea Level '
      f'for this location.')

Runway 30 starts 1.5442948346517866 m above the local Palo Alto geoid.
This is 1.4557051653482134 m off from the published FAA altitude relative to Mean Sea Level for this location.


### D. ENU coordinates of runway's end with the runway's start as the origin

In [20]:
# Obtain the runway end coordinates relative to the runway start in the WGS 84
#     ECEF coordinates
ecef_end_rel_to_start = RWY_30_END_M - RWY_30_START_M

# Perform the rotations to convert to the ENU coordinate system
enu_end_rel_to_start = navutil.enu_from_xyz(
    ecef_end_rel_to_start,
    rwy_30_start_lla[0],
    rwy_30_start_lla[1]
)

print(f'The position of the runway end from the runway start is '
      f'{enu_end_rel_to_start[0]} m East, {enu_end_rel_to_start[1]} m North, '
      f'and {enu_end_rel_to_start[2]} m up.')

The position of the runway end from the runway start is -472.5482654586933 m East, 597.7817327425813 m North, and -0.005550572226179404 m up.


### E. Length of Runway 30 in feet

In [19]:
# Get the length in meters, then convert
len_m = np.sqrt(np.sum(np.square(ecef_end_rel_to_start)))
len_ft = len_m * 3.28084

print(f'Runway 30 is {len_ft} ft long.')

Runway 30 is 2500.0019383229533 ft long.


This is, within very small error, equal to the published length of 2500 ft.

### F. True heading of Runway 30

In [28]:
# Calculate the true heading
rwy_true_heading = np.arctan2(
    enu_end_rel_to_start[0],
    enu_end_rel_to_start[1]
) * navutil.DEG_FROM_RAD

print(f'The true heading of Runway 30 is {rwy_true_heading + 360}°.')

The true heading of Runway 30 is 321.67352296799083°.


This is about 6.6° off from the provided approximate runway heading of 315°.

### G. Gradient of Runway 30

In [30]:
# Calculate the gradient
rwy_30_rise = enu_end_rel_to_start[2]
rwy_30_run = np.sum(np.square(enu_end_rel_to_start[:2]))
rwy_30_gradient = rwy_30_rise / rwy_30_run * 100

print(f'The gradient of Runway 30 is {rwy_30_gradient}%.')

The gradient of Runway 30 is -9.559323741705354e-07%.


This corresponds to the provided value of 0.0%, with significant zeroes to spare!